In [1]:
'''h_DsRnn_Prep.ipynb [++++] Prep all data and train (one) model (per compute node).
calls h_DsRnn_ProcTb.ipynb and h_DsRnn_ExTb.ipynb

* Select one parameter set from ProcTb = Par
* Load data into ExTb according to Par
* Create output folder
* Train and save results

CONF: h_DsRnnAna_Prep_sba.ipynb : Load, prep and save fMRI and phys. data in Gsig.mat files.

PREC: h_DsRnn_Prep_v1b1.ipynb, h_DsRnn_v6b1.ipynb
AUTH: Hendrik.Mandelkow@nih.gov
'''

'h_DsRnn_Prep.ipynb [++++] Prep all data and train (one) model.\ncalls h_DsRnn_ProcTb.ipynb and h_DsRnn_ExTb.ipynb\n\n* Select one parameter set from ProcTb = Par\n* Load data into ExTb according to Par\n* Create output folder\n* Train and save results\n\nCONF: h_DsRnnAna_Prep.ipynb : Load, prep and save fMRI and phys. data in Gsig.mat files.\n\nPREC: h_DsRnn_Prep_v1b1.ipynb, h_DsRnn_v6b1.ipynb\nAUTH: Hendrik.Mandelkow@nih.gov\n'

### TODO:
 - RNN:
   - [ ] use corr.dist as loss
   - [ ] use GRU (with lin. act.) as output
   - [ ] use small layer just before output
 - [ ] train on diff(MRI) or piecewise const.
 - [ ] compare inter-scan, inter-session, inter-subj regression of Gsig
 - [ ] compare val.su and sections with w/o drop
 - [ ] export & plot layers
 - [x] sort ValSu by size
 - [ ] Dropout full TRs
 - [ ] Try different MaskVal eg 0 ((mean)) or const.
   - [ ] read on LSTM missing values
 - [ ] Add more subjects
 - [ ] see TODO in h_DsRnn_v4a5.ipynb

### DONE:
 - [x] Let's not save ExTb.
       - PD df.to_hdf5 doesn't work bc of mixed data types
       - Pickle is an option


### How to record processing parameters (ProcPar) and other metadata?

 1. Save ExTb, ProcTb, etc and re-load for analysis.
    * Requires processing twice!
 2. Create identical(!) ExTb, ProcTb for both training and analysis.
    * Does not require separate processing.
    * May still want to save for reference.


## sbatch


---------------------------------------------------------------------------------
```bash
# cd /data/mandelkowhc/Sleep/0302_Proc06_3su/sb_tmp

cd ~/matlab/htools1/hpy # where to put slurm-*.out and script_1234.ipynb
NBCOPT="--clear-output --ExecutePreprocessor.timeout=None --TemplateExporter.exclude_output=False --TemplateExporter.exclude_markdown=False --TemplateExporter.exclude_raw=True "
# --to notebook # ?!?

NBCIN=~/matlab/htools1/hpy/h_DsRnn_Prep_sba4
NARR=0-55
echo "#! /bin/bash
jupyter nbconvert $NBCOPT --output ${NBCIN##*/}_\$SLURM_ARRAY_JOB_ID-\$SLURM_ARRAY_TASK_ID --execute $NBCIN.ipynb" \
| sbatch -a $NARR -J ${NBCIN##*/} -t 36:00:00 --mem=64g -c 14 -p gpu --gres=gpu:p100:1,lscratch:32

```


# Imports

In [2]:
%run h_DsRnn_def.ipynb

[NbConvertApp] Converting notebook htools_v1b.ipynb to python
[NbConvertApp] Writing 34921 bytes to htools_v1b.py


   h_DsRnn_def.ipynb:15


# Proc Parameters [`Par` / `ProcTb`]

In [3]:
%run h_DsRnn_ProcTb.ipynb
ProcTb.shape

(38, 471)

In [ ]:
ProcTb = ProcTb.filter(like='Su1Vx7Lxy')
#ProcTb = ProcTb.filter(like='Su0Vs1Lxy')
#ProcTb = ProcTb.filter(like='Su0Vs1Rper')
#ProcTb = ProcTb.filter(like='Su1Vx7Rper')
#ProcTb = ProcTb.filter(like='Su1Vx7Drop')
#ProcTb = ProcTb.filter(like='Su0Vs1')
#ProcTb = ProcTb.filter(like='Su0Vs1Rx1s')
#ProcTb = ProcTb.filter(like='Su0Vs1Rs')
# ProcTb = ProcTb.filter(like='Su1Vf')
#ProcTb = ProcTb.filter(like='Vs1Xn')
#ProcTb = ProcTb.filter(like='Vx1Xn')
# ProcTb = ProcTb.filter(like='Su1Vr1Xn')
# ProcTb = ProcTb.filter(like='Su1Vr3')
# ProcTb = ProcTb.filter(like='Su1Vr3linNt30')
# ProcTb = ProcTb.filter(like='Su1Vr3linTbat8')
print('Run sbatch array with NARR=0-%u'%(ProcTb.shape[-1]-1))

In [ ]:
ProcTb.T.reset_index(drop=True).head()
# ProcTb.T.iloc[1]
#ProcTb.head()


## sbatch processing?
Select Par = ProcPar.iloc[ :, SLURM_ARRAY_TASK_ID]

In [ ]:
### Assign Par.SbJob and .SbTask
SbJob = os.environ.get('SLURM_ARRAY_JOB_ID')
SbTask = os.environ.get('SLURM_ARRAY_TASK_ID')
if SbJob is not None:
    SbTask = int( SbTask )
else:
    SbJob = '000'
    SbTask = 0

# Par = ProcTb.iloc[:,SbTask] # +++ # want .to_dict()???
Par = ProcTb.iloc[:,SbTask].to_dict() # +++ # want .to_dict()???
Par = ddict(Par)
Par.SbTask = SbTask
Par.SbJob = SbJob
# del SbJob, SbTask


## ProcDir: set output folder
Might as well do this 1st to check for existing.

Let the ProcPath always be *DataDir/Proc/*. Rename older folders to Proc01,02,...

In [ ]:
# Let the ProcPath always be *DataDir/Proc/*. Rename older folders to Proc01,02,...
os.makedirs( Par.DataDir+'Proc',exist_ok=True) # cleate and cd to DataDir/Proc
os.chdir( Par.DataDir+'Proc' ) # ***
os.getcwd()


In [ ]:
import shutil
# Par.ProcDir = 'Km_%s-%02u_%s/'%( Par.SbJob[-3:], Par.SbTask, Par.ProcId )
assert len( Par.ProcId.split('_') ) > 1, 'Oops?! Improper ProcId e.g. A3_Par1Par2...'
Par.ProcDir = 'Km_%s/'%Par.ProcId # +++
# print( Par.ProcDir )
shutil.rmtree('Km_000_TEST',ignore_errors=True) # avoid error below
os.makedirs( Par.ProcDir, exist_ok=False) # +++ Stop execution on existing dir
# if SbJob is not None:
#     assert not os.path.exists( Par.ProcDir ), 'Oops! Output folder ProcDir already exists!'
print(os.getcwd()+'/'+Par.ProcDir)


# Load data - global signal

In [ ]:
# Note: ProcId s are now expected to have this format: A12_Par1Par2Par3 (Nr_Description)
Par.ProcId.split('_')

In [ ]:
%%time
# ExTb: Table (PD Df) of experiments / data files.
%run ~/matlab/htools1/hpy/h_DsRnn_ExTb.ipynb

## How to save ExTb:
 1. don't save, recreate
     * prob. best, since tra / val data may be different 
 2. pickle
 3. split off data arrays
 4. Xarray
 5. list of dict
 6. alternative ot PD table
 

# mk TrainGen
 * AllData = train. and val. data
 * Data = training data
 * ValData = validation data
 
Since TraGen and ValGen have to be the same size for a *stateful* RNN, it might be convenient simply to pass AllData along with separate masks for Data and ValData. (For added *security* one might zero training / validation data to guard against a buggy leak!?)


In [ ]:
# TODO: Use SecTb instead?
AllData = np.concatenate( ExTb.Data.tolist(), 0) # [Sec,t,Ch] # AllData = Train+Val data
AllData_ValTag = np.concatenate( ExTb.ValSec.tolist(), 0) # [Sec] boolean index

if Par.ValFrac: # +++
    # AllData_ValTag[ AllData_ValTag < 1 ] = AllData_ValTag[ AllData_ValTag < 1 ] >= (1-Par.ValTag) # +++
    AllData_ValTag = np.where( AllData_ValTag<1, AllData_ValTag>=(1-Par.ValFrac), AllData_ValTag )

AllData_ValTag = AllData_ValTag.astype(int)

#AllData = np.concatenate( SecTb.Data.tolist(), 0)
#AllData_ValTag = np.concatenate( SecTb.ValSec.tolist(), 0)

AllData = AllData[...,[0,1,-2,-1]]
AllData.shape

In [ ]:
if Par.RandPerm:
    warn('+++ Randomized control experiment!')
    # AllData = np.concatenate( [ np.random.permutation( AllData[...,:2]), AllData[...,2:] ], -1) # simpler!
    # np.random.shuffle( AllData[...,:2]) # simplest?!

    tmp = np.random.permutation( AllData.shape[0])
    while np.any( tmp == np.arange(tmp.size)):
        tmp = np.random.permutation(tmp)

    AllData = np.concatenate( [ AllData[tmp,:,:2], AllData[...,2:]], -1)
    AllData.shape

In [ ]:
#ValData = AllData[ AllData_ValTag != 0 ]
#ValData = np.copy(AllData[AllData_ValTag != 0]) # +++ copy necessary?
Data = AllData[ AllData_ValTag == 0 ] # +++ training data
#> Data = AllData[ AllData_ValTag < (1-Par.ValFrac) ] # +++ training data
#> ValData = AllData[AllData_ValTag != 0] # +++ validation data
#> ValData = AllData[AllData_ValTag == Par.ValTag] # +++ validation data 1,2,3 = ValMiddle, ValEx1, ValSu
#< ValData = AllData[ np.any(AllData_ValTag.reshape(-1,1)==np.array(Par.ValTag).reshape(1,-1),1)]
ValData = AllData[ np.any( np.c_[AllData_ValTag] == np.r_[Par.ValTag],1) ] # +++

assert Data.size>0, 'Oops! No training data!'
assert ValData.size>0, 'Oops! No ValData!'

Data.shape, ValData.shape

In [ ]:
NT = Par['NT']
NY = 1 # ***
NX = Data.shape[-1]-NY
assert (Par.NX==NX) and (Par.NY==NY), 'Oops!'
NB = Data.shape[0]
# TRFS = Par.TRFS
TGEN = {}
TGEN['Drop'] = Par.Drop
TGEN['Xtrafo'] = Par.Xtrafo
#< TGEN['RandXlead'] = Par.RandXlead
# TrainGen = hBatchSeq1y( Data.reshape(-1,Data.shape[-1]), NT=NT, NX=NX, NB=NY*NB, Drop=0)
TrainGen = hBatchSeq1y( Data.reshape(-1,Data.shape[-1]), NT=NT, NX=NX, NB=NY*NB, **TGEN)
TrainGen.RandXlead = Par.RandXlead
TrainGen.RandYscale = Par.RandYscale
TrainGen[0][0].shape

In [ ]:
# NOTE: Need to append 0s to make ValData the same (batch) size as Data for stateful RNN model.
# VGEN['Mask'] shall be used to mask the excess ValData.
VGEN = {}
VGEN['Drop'] = Par.ValDrop
VGEN['Xtrafo'] = Par.Xtrafo
# Need to pass a sample_weights, in order to have ValidGen return (X,Y,W).
# Weights used here to mask smaller val.batch size.
VGEN['Mask'] = (np.arange(Data.shape[0])<ValData.shape[0]).astype(int) # mask of ones. see above
# ValData = np.copy(ValData)
# ValData.resize(Data.shape) # append zeros. Doesn't work. Stupid Python.
tmp = ValData
ValData = 0*Data
ValData[:tmp.shape[0]] = tmp
# ValData = np.resize( ValData, Data.shape) # repeat array
ValData.shape


In [ ]:
# ValidGen = hBatchSeq1y( ValData.reshape(-1,ValData.shape[-1]), NT=NT, NX=NX, NB=NY*NB, Drop=Par.ValDrop, Mask=Mask)
ValidGen = hBatchSeq1y( ValData.reshape(-1,ValData.shape[-1]), NT=NT, NX=NX, NB=NY*NB, **VGEN)


******************************************
## mk ValGen
Since TraGen and ValGen have to be the same size for a *stateful* RNN, it might be convenient simply to pass AllData along with separate masks for Data and ValData. (For added *security* one might zero training / validation data to guard against a buggy leak!?)


# RNN model

## Create RNN

In [ ]:
TRFS = Par.TRFS
BPE,NT,NY = TrainGen[0][1].shape
NX = TrainGen[0][0].shape[-1]-NY
NB = BPE//NY

In [ ]:
NB, BPE, NT, NX, NY

In [ ]:
# [ exec(n+'=Par["%s"]'%n ) for n in 'TR Fs'.split() ] # Why not work?!?
# TR,Fs,NT,NX,NY,NB = [ Par[n] for n in 'TR,Fs,NT,NX,NY,NB'.split(',') ]
if not isinstance(NT,int) or (NT < TRFS): # NT given in sec
    NT = int(NT//TR*TR*Fs) # each seq.length in samples (Fs)
    Par.update(NT=NT)

# Prob.WRONG! assert Data.shape[1]>=NX+NB, 'Oops, not enough data channels!'
assert Data.shape[1]>=NX+NY, 'Oops, not enough data channels!'

keras.backend.clear_session()
#< Par['RnnArg'][0] = []
# RNN = mkRNN( **Par['RnnArg'] ) # +++ ***
if True:
    # RNN = mkRNN( [NX+1,'G',2**6,2**6,2**5,'D',1], NT, BPE) # +++ ***
    Par['RnnNio'][0] = NX+1
    # RNN = mkRNN( Par['RnnNio'], NT, BPE) # +++ ***
    Par['RnnArg'] = { 'Nio':Par['RnnNio'], 'Nsteps':NT, 'Nbatch':BPE}
    RNN = mkRNN( **Par['RnnArg'] ) # +++ ***
else:
    Par['RnnArg'] = { 'Nio':Par['RnnNio'], 'Nsteps':NT, 'Nbatch':BPE, 'stateful':False}
    # RNN = mkRNN( [NX+1,'G',2**6,2**6,2**5,'D',1], NT, BPE, stateful=False) # +++ ***
    RNN = mkRNN( **Par['RnnArg'] ) # +++ ***
    warn('+++ TEST TEST TEST +++ stateful = False!!!')

# Par['RnnArg'] = RNN.mkRNNargs # FIXIT: Broke with tf.keras

if False:
    # HOWTO change Keras model by recompiling:
    # HOWTO use tf.keras.losses.CosineSimilarity():
    RNN.compile( loss=tf.keras.losses.CosineSimilarity(axis=-2), optimizer='adam', metrics=[ KCustoms['hWRVF'] ]) # +++

# RNN.mkRNNargs = [[NX+1,'G',2**6,2**6,2**5,'D',1], NT, NY*NB]
RNN.summary()

In [ ]:
try: Par['Ngpus'] = os.environ.get('SLURM_STEP_GPUS').count(',')+1
except: Par['Ngpus'] = 0
if Par['Ngpus']:
    warn('You *should* be using Keras..multi_gpu_model()!!!')

## make data generators: TrainGen, ValidGen, (TestGen)
### The validation batch size problem
In Keras a *stateful* RNN *requires* a fixed training batch size (`NB*NY` is not None). This in turn mandates that both TrainGen and ValidGen have the same `NB*NY`. For continuous (stateful) prediction (NB=1) we must define a separate model (RNNp) *and* data generator (TestGen).

# Callbacks (FITPAR)

In [ ]:
#< os.makedirs( Par.DataDir+'Proc',exist_ok=True)
os.chdir( Par.DataDir+'Proc' ) # *** Just to be sure


In [ ]:
#%%
ProcDir = Par.ProcDir
FITPAR = {'callbacks':[]}
# FITPAR['epochs'] = 1
# FITPAR['validation_split'] = 0.2 # v.split not for generator
# https://keras.io/callbacks/#modelcheckpoint
# tmp = re.sub(r'[-_.][^/\\]+$',r'-e{epoch:02d}-{loss:.2f}.h5',ModelFile)
#> tmp = re.sub(r'\.[^/\\]+$',r'_Ep{epoch:03d}.h5',ModelFile)
# tmp = 'tmp.h5'
# keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
FITPAR['callbacks'] += [keras.callbacks.ModelCheckpoint(ProcDir+'model.h5', monitor='loss')]; # print(ModelFile)
FITPAR['callbacks'] += [keras.callbacks.ModelCheckpoint(ProcDir+'model_BestTra.h5', monitor='loss',save_best_only=True)]
FITPAR['callbacks'] += [keras.callbacks.ModelCheckpoint(ProcDir+'model_BestVal.h5', monitor='val_loss',save_best_only=True)]

In [ ]:
os.makedirs( ProcDir+'weights/',exist_ok=True)
FITPAR['callbacks'] += [keras.callbacks.ModelCheckpoint(ProcDir+'weights/weights_E{epoch:05d}.h5', monitor='loss',save_best_only=True,save_weights_only=True)]

In [ ]:
pwd

In [ ]:
FITPAR['callbacks'] += [ hResetStatesCb() ] # False/True on_epoch_begin / _end
# KmInfoStr.append('hResetStatesCb(False): on_epoch_begin')

In [ ]:
# FITPAR.update({'validation_data': ValidGen, 'validation_steps': 32, 'validation_freq': 8})
# NOTE: val_freq requires Keras >2.2?!
# FITPAR.update({'validation_data': ValidGen, 'validation_steps': 32})
FITPAR.update({'validation_data': ValidGen, 'validation_steps':None})


In [ ]:
# NOTE: Many optimizers include adaptive LR. Therefore LR schedule may be less irrelevant.
# TODO: is min_delta dependent on the abs scale of loss?
# NOTE: ?!? Reduce LR, if loss hasn't fallen by at least min_delta over the past "patience" epochs?!
# Basically, require average dLoss/dt > min_delta / patience
# keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
# FITPAR['callbacks'] += [keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.8, patience=10, min_lr=0.00001, verbose=1)]
FITPAR['callbacks'] += [keras.callbacks.ReduceLROnPlateau(monitor='hWRVF', min_delta=0.005, patience=8, factor=0.8, min_lr=0.00001, cooldown=0, verbose=1)]


In [ ]:
FITPAR['callbacks'] += [keras.callbacks.CSVLogger(ProcDir+'train_log.csv',separator=',',append=True)]


In [ ]:
# keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)


# Load model?

******************************************************************
# Save model and training parameters

In [ ]:
os.chdir(Par.DataDir+'Proc/')
# This should be done before.
# os.makedirs(ProcDir, exist_ok=False) # +++ error if exist, no overwrite!
print(os.getcwd()+'/'+ProcDir)

### save metaparameters (Proc)Par

In [ ]:
hdf5.savemat( ProcDir+'metapar.mat', Par, truncate_existing=True)
#< scipy.io.savemat( ProcDir+'metapar.mat', Par) # cannot store None?!?


In [ ]:
import json
with open( ProcDir+'metapar.json', 'w') as file:
    json.dump( Par, file, indent=4, sort_keys=True, separators=(', ',': '))


In [ ]:
import yaml
with open( ProcDir+'metapar.yaml', 'w') as file:
    yaml.dump( dict(Par), file)
    # yaml.dump( Par, file, indent=4, sort_keys=True, separators=(', ',': '))


### save RNN model architecture

In [ ]:
with open( ProcDir+'model.json', 'w') as file:
    file.write( RNN.to_json() )


### save training and validation data

In [ ]:
### Save SecTb to .h5
#< ProcDir = ''
SecTb['Data0'] = SecTb['Data'].apply(lambda x: x.flat[0])
SecTb.drop(columns='Data').to_hdf( ProcDir+'SecTb_xs.h5','SecTb') # ?!? Pandas BUG! must use format='table'

### save TensorBoard training stats

# Train Model

In [ ]:
%notebook $ProcDir/h_DsRnn_tmp.ipynb

In [ ]:
# hist = RNN.fit_generator(TrainGen, initial_epoch=n, epochs=n+1, **FITPAR) # initial / final epochs
hist = RNN.fit_generator(TrainGen, epochs=Par['Epochs'], **FITPAR) # initial / final epochs
# NOTE: hist = model.fit() = model.history.history # which is replaced at every call to fit()
# If hist = fit() is interrupted, hist is empty, but model.history.history still exists.

## Retrain model successively on ValSu + ValEx1 + ValMiddle

*******************************************************
# Load best model

In [ ]:
par = Par

In [ ]:
# K.clear_session() # necessary?!?
print('+ Load '+par['ProcDir']+'model_BestVal.h5')
if False:
    par.update( RNN = keras.models.load_model( par['ProcDir']+'model_BestVal.h5', custom_objects=KCustoms))
elif False:
    with open(ProcDir+'model.json','r') as json_file:
        RNN = keras.models.model_from_json( json_file.read() )
    RNN.load_weights(ProcDir+'model_BestVal.h5')
else:
    RNN.load_weights(ProcDir+'model_BestVal.h5')

## Eval on val.data

In [ ]:
## Predict yh and append x,y,yh to par[]
par = { 'x': ValidGen.getX(), 'y': ValidGen.getY(), 'yh': ValidGen.predict( RNN ) }

## Reshape x,y,yh to [Sec,t,Ch] and discard dummy channels ValidGen.Mask==0
tmp = ( ValidGen.NB, len(ValidGen)*ValidGen.NT, ValidGen.NY )
par['x'] = par['x'].reshape( *tmp[:-1], -1)[ValidGen.Mask>0,...]
par['y'] = par['y'].reshape( *tmp )[ValidGen.Mask>0,...]
par['yh'] = par['yh'].reshape( *tmp )[ValidGen.Mask>0,...]


## Save val. results as ???
EvalData, ValOut, ProcData?


In [ ]:
# tmp = SecTb.drop('Data').to_dict
par['SecTb'] = SecTb.to_dict('list')


In [ ]:
# SecTb[['Su','Se','Ex','Sec','ValSec','Data0']].to_numpy()
par['SecNp'] = SecTb[['Su','Se','Ex','Sec','ValSec']].to_numpy()


In [ ]:
assert ProcDir[-1]=='/', 'Hoppla!?!'
hdf5.savemat(ProcDir+'EvalData.mat', dict(par), truncate_existing=True, format='5')


## TODO: Predict internal layer activations
see `h_DsRnnAna_v6b2_sba` RNNa.pop

# Loop over weights and val. sets
## TODO:
 - [ ] use .predict instead of .evaluate?!
 - [ ] use masked ValData with original model?!
 - [ ] use mkRNN from scratch?!
 - [ ] forget about this and simply retrain?!
 